# Spikes and neurons: Waveforms

In this notebook, you will study spike waveforms, and explore whether these tell you anything about cell types and recording configuration.

## Prerequisites

We load the same data as before. Nothing too surprising here.

In [ ]:
!pip install ephysio

In [ ]:
from ephysio import openEphysIO
from ephysio import kilosortIO
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root = "/content/drive/MyDrive/datasai-daw/data/2021-07-20_11-59-01"

In [ ]:
#%% Load experiment data and events
oe = openEphysIO.Loader(root)
strm = oe.spikestream(0)
evts = oe.nidaqevents(strm, rec=9)

#%% Load spikesorted data from kilosort
ks = kilosortIO.Reader(oe.contfolder(oe.spikestreams()[0], rec=9) + "/kilosort20_output")
spks = ks.spikesbycluster('good')

## Organizing the data

As we saw this morning, there are many ways to represent the same data. Here, we will store all our findings in the columns of a pandas "dataframe". Whether you prefer this over other methods is very much a question of personal style. Dataframe syntax can look a little overwrought at first, but they might grow on you, so hang on, and keep an open mind!

In [ ]:
# Extract the data and construct lists
celid = []
electrodes = []
keyelectrode = []
template = []
keytipdist = []
tipdists = []
counts = []
for clsid in spks.keys():
    celid.append(clsid)
    elcs = ks.electrodesforcluster(clsid)
    electrodes.append(elcs)
    keyelectrode.append(elcs[0])
    counts.append(len(spks[clsid]))
    keytipdist.append(ks.tipdist_unit(clsid))
    tipdists.append(ks.tipdist_electrode(elcs))
    tmpl = ks.templateforcluster(clsid)
    tmpl = [tmpl[:, ks.elc2chan[elc]] for elc in elcs]
    template.append(np.array(tmpl).T)

# Combine them into a Panda dataframe ʕ•ᴥ•ʔ
waves = pd.DataFrame({  "celid": celid,
                        "electrodes": electrodes,
                        "keyelectrode": keyelectrode,
                        "keytipdist": keytipdist,
                        "tipdists": tipdists,
                        "template": template,
                        "counts": counts
                       })

Based on previous analysis, we have determined that 5 cells in this recording are antidromically activated by the stimulation electrode in V1. These are therefore feedback cells, and could be of special interest:

In [ ]:
antidromic = [231, 293, 314, 316, 524]

(If you are interested, you can, later this afternoon, study the responses of these and other cells to electrical stimuli, and see if you are convinced by our reasoning.)

We also concluded that a dozen or so cells responded synaptically to electrical stimulation in V1. These are therefore feedforward input cells, and could also be of special interest:

In [ ]:
orthodromic = [49, 119, 227, 243, 250, 251, 293, 302, 314, 316, 377, 395, 524, 535, 592]

We know nothing about the type of all the other cells.

### Thought exercise

Based on our electrical stimulation experiment, is it fair to conclude that the remaining cells are *not* feedforward and *not* feedback cells?

Based on doing this exercise ourselves, we believe that for these two cells we recorded from the axon rather than from the soma:

In [ ]:
axonal = [231, 49]

which is not to say that we recorded somatically from all the others. See below.

Let's add cell type information to the dataframe:

In [ ]:
# Define a function to determine cell type
def get_celtype(celid):
    typ = 'other'
    if celid in antidromic:
        typ = "anti"
    elif celid in orthodromic:
        typ = "ortho"
    if celid in axonal:
        typ += "-axonal"
    return typ

# Apply the function to create the celtype column
waves["celtype"] = waves["celid"].apply(get_celtype)

### Exercise: Spike waveforms

For each of the putative neurons, the dataframe (and the `template` list) contains the average waveform recorded for that neuron's spikes on several electrodes. Plot these average waveforms for a couple of units in separate graphs. Do all neurons have similar waveforms on their primary electrodes (first column of the template)?

In [ ]:
# Insert your code here


### Exercise

Plot the waveforms (on the principal electrode) for the antidromic neurons and the orthodromic neurons. Do they look different? Would you expect them to?

## Classifying waveforms

We have 228 neurons, so looking at all their waveforms gets a bit much. Perhaps dimensionality reduction can help us to summarize waveform shape into a small number of parameters that we can more easily visualize.

Let's start with PCA, and let's store the results in the dataframe.

In [ ]:
#%% Apply PCA to waveforms
ncomp = 10
pca = PCA(ncomp)
templPC = pca.fit_transform(templates)

waves["templatePCA"] = waves.apply(lambda row: templPC[row.name], axis=1)

Let's make a plot with the first PC on the x-axis and the second on the y-axis, and let's color code the cells by their connection type (antidromic, orthodromic, unspecified):

In [ ]:
# Helper function for color coding, will be useful later
def cellcolor(celtype):
    if celtype.startswith("anti"):
        return 'r'
    elif celtype.startswith("ortho"):
        return 'b'
    else:
        return [.7,.7,.7]

In [ ]:
plt.figure()
for k, row in waves.iterrows():
    x = row.templatePCA[0]
    y = row.templatePCA[1]
    c = cellcolor(row.celtype)
    if row.celtype.endswith("axonal"):
        s = 'x'
    else:
        s = '.'
    plt.plot(x, y, s, color=c)

### Exercise

What does it mean that the first PC is bimodally distributed? Plot a few templates from the left camp and from the right camp. Do you think these are genuinely different populations? Also, explore a few more PCs to see if they help cluster the data.

We think that the PCA results were less enlightening than we had hoped. But we do see that there appear to be different kinds of waveforms: Those that are mainly monophasic with shallow side lobes, and those that are strongly biphasic or triphasic. Let's manually explore waveforms a little.

First step, the data were sampled at 30 kHz. That's not bad, but the waveforms would look cleaner if we upsampled a little. That way, we can better measure waveform shape. In reality, this should be done inside the spike sorting step, to calculate spike timing at even higher temporal resolution, and get cleaner templates, but that is beyond the scope of today's exercise.

In [ ]:
# This function takes a signal X and interpolates it by a factor N using
# "Fourier upsampling", which does not add any spurious high-frequency
# components to the signal.
def fupsample(x, n):
    L = len(x)
    H = L // 2
    f = np.fft.fft(x[:2 * H])
    f = np.concatenate((f[:H],
                        np.zeros((H * 2 * (n - 1)), f.dtype),
                        f[H:2 * H]), 0)
    y = np.real(np.fft.ifft(f)) * n
    return y

In [ ]:
# This function displaces data by a random amount between -0.5 and +0.5, to
# avoid the appearance of spurious periodicity in digitally sampled data.
def jitter(x):
    return x + np.random.random(x.shape)-.5

The following is a very ugly piece of code that calculates the width of the main peak in a template, as well as the width of the secondary peak (the largest peak in the opposite direction), and the latency between the main and secondary peaks.

In [ ]:
plt.figure()
xx = []
yy = []
for k, row in waves.iterrows():
    tmpl = row.template[:,0]
    tmpl = tmpl[25:]
    tmpl = fupsample(tmpl, 4)
    tmpl = tmpl[4:-4]
    x0 = np.argmax(np.abs(tmpl))
    tmpl = tmpl / tmpl[x0]
    x1 = np.argmax(tmpl[x0:] < .2)
    x2 = np.argmax(tmpl[x0::-1] < .2)
    dx = x1 + x2 # Width of main peak
    x3 = np.argmin(tmpl)
    dy = x3 - x0 # Dist. b/w main peak and greatest opposite peak
    tmpl = tmpl / tmpl[x3]
    x1 = np.argmax(tmpl[x3:] < .2)
    x2 = np.argmax(tmpl[x3::-1] < .2)
    dz = x1 + x2 # Width of opposite peak
    c = cellcolor(row.celtype)
    xx.append(np.abs(dy)/30/4)
    yy.append(np.abs(dz)/30/4)
    if row.counts < 1000:
        # Let's not plot neurons with very low firing rates (1000/hour ≅ 0.28 Hz)
        continue
    plt.plot(1000 * np.abs(jitter(dy)) / 30 / 4, 1000 * np.abs(jitter(dz) / 30 / 4), '.', color=c)

x = np.arange(400)
plt.plot(x, 400-x)
plt.xlabel("Latency between main and secondary peak (us)")
plt.ylabel("Width of secondary peak (us)")

waves["xx"] = xx
waves["yy"] = yy

### Exercise

The above piece of code, written late at night, commits several cardinal sins. Two of the main ones are:

* It combines calculation and plotting in one loop
* It uses completely meaningless variable names

Can you fix it up?

In [ ]:
# Insert your code here


Regardless of the ugliness of the code, this plot suggests that there are some neurons with much faster spikes then others. The blue line is my attempt to manually separate the clusters.

Let's plot the preceding PCA results, but use dots and x-es to indicate the neurons above and below the separating line.

In [ ]:
plt.figure()
for k, row in waves.iterrows():
    if row.counts < 1000:
        continue
    x = row.templatePCA[0]
    y = row.templatePCA[1]
    c = cellcolor(row.celtype)
    if row.yy < 0.4 - row.xx:
        s = 'x'
    else:
        s = '.'
    plt.plot(x, y, s, color=c)

Let's plot the raw templates with the same organization.

In [ ]:
f, ax = plt.subplots(3, 2)
for k, row in waves.iterrows():
    if row.counts < 1000:
        continue
    x = row.templatePCA[0]
    y = row.templatePCA[1]
    if row.celtype.startswith("ortho"):
        r = 0
    elif row.celtype.startswith("anti"):
        r = 1
    else:
        r = 2
    if row.yy < 0.4 - row.xx:
        c = 0
    else:
        c = 1
    ax[r,c].plot(row.template[25:,0])

In the plots above, the top row are the orthodromics, second row antidromics, bottom row unspecified. Left column are templates found below the blue separating line, right column above.

I find these results less than satisfying. But I still have the intuition that there are "fast" and "slow" templates. Perhaps we can get better results in frequency space? Let's Fourier transform the templates:

In [ ]:
sqrpwrs = []
for k, row in waves.iterrows():
    tmpl = row.template[:,0]
    tmpl = tmpl[25:]
    fup = fupsample(tmpl, 4)
    amp = fup.max() - fup.min()
    tmpl = tmpl/amp
    ftmpl = np.fft.fft(tmpl - np.mean(tmpl))
    pwrspec = np.abs(ftmpl**2)
    sqrpwr = np.sqrt(pwrspec)
    sqrpwrs.append(sqrpwr)

Here, I first normalized each template by its peak-to-peak amplitude, then calculated the Fourier transform, and calculated the square root of the power as a function of frequency.

### Exercise

Take a look at a few spectra. Do you find them illuminating?

In [ ]:
# Insert your code here


The spectra are also too much to all take in. How about we do PCA on the spectra? After all, the spectra are simply vectors, just as the templates themselves.

### Exercise

Perform the PCA on the spectra, exactly analogously as before, and store the results in the dataframe, in a new column called "fourierPCA".

In [ ]:
# Insert your code here


### Exercise

Make a plot of the second vs the first spectrally defined PC, similar to the one we made for the temporal PCs.

In [ ]:
# Insert your code here


Learn anything useful? Any other ideas how to algorithmically understand the different waveforms?

In [ ]:
# Optionally insert code here


### Final exercise

So far, we have mainly looked at the first template for each cell. But our dataframe (and the `template` variable) contains templates recorded from all electrodes that pick up a signal from a given cell. Select a few cells of interest (e.g., antidromic vs orthodromic, or “fast” vs “slow”) and plot the different templates for a given cell together. If a cell is “fast” (or “slow”) on one electrode, is it necessarily also fast (or slow) on the other electrodes? How might we interpret this?

In [ ]:
# Insert your code here
